In [1]:
import pandas as pd
import numpy as np
import requests
import re
import json
from bs4 import BeautifulSoup

In [20]:
def get_snap_count(position='qb'):
    """_summary_

    Args:
        position (int, optional): the position to scrape [qb, rb, wr, te, dl, lb, db]. Defaults to qb.

    Returns:
        Beautifulsoup: the Beautifulsoup scraped object (html)  
    """
    url = f'https://www.fantasypros.com/nfl/reports/snap-counts/{position}.php'
    params = {
        'show': 'perc',
    }
    header = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"
    }
    r = requests.get(url, headers=header, params=params)
    print(f"{r} - Scraped {url}")
    soup = BeautifulSoup(r.text, 'html.parser')
    div_table = soup.find('div', {'class': 'mobile-table'})
    if div_table:
        scrape_res = pd.read_html(str(div_table))
        df = scrape_res[0]
        df.insert(2, column='Postion', value=position.upper())
        for c in ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', 'AVG']:
            if df[c].notnull().all():
                df[c] = pd.to_numeric(df[c].str.replace("%", ""))
        return df
    else:
        return None
    
data = get_snap_count(position="db")
print(data)
data.info()

<Response [200]> - Scraped https://www.fantasypros.com/nfl/reports/snap-counts/db.php
                Player Team Postion    1    2    3    4   5   6   7  ...  11  \
0       Trent McDuffie   KC      DB  100   99   82   98 NaN NaN NaN  ... NaN   
1           Bryan Cook   KC      DB  100   99  100   83 NaN NaN NaN  ... NaN   
2       L'Jarius Sneed   KC      DB  100  100   82  100 NaN NaN NaN  ... NaN   
3         Mike Edwards   KC      DB   23   31   41   44 NaN NaN NaN  ... NaN   
4      Joshua Williams   KC      DB   56   46   33   47 NaN NaN NaN  ... NaN   
..                 ...  ...     ...  ...  ...  ...  ...  ..  ..  ..  ...  ..   
282       Dean Marlowe  LAC      DB    0    0    0  100 NaN NaN NaN  ... NaN   
283  Kendall Sheffield   SF      DB    0    0    0   15 NaN NaN NaN  ... NaN   
284      Ji'Ayir Brown   SF      DB    0    0    0   15 NaN NaN NaN  ... NaN   
285        Jamal Adams  SEA      DB    0    0    0   12 NaN NaN NaN  ... NaN   
286         Bryce Hall  NYJ      D

In [22]:
df_list = list()
for p in ['qb', 'rb', 'wr', 'te', 'dl', 'lb', 'db']:
    df = get_snap_count(position=p)
    df_list.append(df)
data = pd.concat(df_list, ignore_index=True)
data.to_csv('./scraped_season_2023/snap_count.csv', index=False)

<Response [200]> - Scraped https://www.fantasypros.com/nfl/reports/snap-counts/qb.php
<Response [200]> - Scraped https://www.fantasypros.com/nfl/reports/snap-counts/rb.php
<Response [200]> - Scraped https://www.fantasypros.com/nfl/reports/snap-counts/wr.php
<Response [200]> - Scraped https://www.fantasypros.com/nfl/reports/snap-counts/te.php
<Response [200]> - Scraped https://www.fantasypros.com/nfl/reports/snap-counts/dl.php
<Response [200]> - Scraped https://www.fantasypros.com/nfl/reports/snap-counts/lb.php
<Response [200]> - Scraped https://www.fantasypros.com/nfl/reports/snap-counts/db.php
